In [3]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import confusion_matrix

def rmse(y, y_hat):
    y = np.array(y)
    y_hat = np.array(y_hat)
    return np.sqrt(np.mean((y - y_hat) ** 2))


def report_results_metrics(results_path):
    results = pd.read_csv(results_path, sep=";")
    cols = ['project_id', 'real_votes', 'real_rank', 'predicted_votes']
    results = results.filter(cols)

    y_test = results['real_votes'].values
    y_hat = results['predicted_votes'].values
    
    tau, p_value = kendalltau(y_test, y_hat)

    print(f"RMSE: {rmse(y_test, y_hat):.4f}")
    print(f"Kendall-thau: {tau:.4f}, (p-value: {p_value:.4f})")


def report_metrics(result_path, top_k=5):
    
    results = pd.read_csv(result_path, sep=";")
    cols = ['project_id', 'real_votes', 'predicted_votes']
    results = results.filter(cols)

    top_k_real_ids = results.nlargest(top_k, 'real_votes')['project_id'].tolist()
    top_k_pred_ids = results.nlargest(top_k, 'predicted_votes')['project_id'].tolist()

    results['is_real_topk'] = results['project_id'].isin(top_k_real_ids).astype(int)
    results['is_predicted_topk'] = results['project_id'].isin(top_k_pred_ids).astype(int)

    #top_k sets
    topk_real_ids = set(results.nlargest(top_k, 'real_votes')['project_id'])
    topk_pred_ids = set(results.nlargest(top_k, 'predicted_votes')['project_id'])


    cm = confusion_matrix(results['is_real_topk'], results['is_predicted_topk'])
    TN, FP, FN, TP = cm.ravel()

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # Jaccard Calculation
    intersection = topk_real_ids & topk_pred_ids
    union = topk_real_ids | topk_pred_ids
    jaccard_index = len(intersection) / len(union) if len(union) > 0 else 0.0

    return {
        "top_k": top_k,
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1_score,
        "jaccard_index": jaccard_index
    }

name = 'wrc_pva_pca20_full_1'
results_path = f'output/predictions/{name}.csv'
report_results_metrics(results_path)
print("##")
data = []
for i in range(15):
    data.append(report_metrics(results_path, top_k=i+1))

top_k_ds = pd.DataFrame(data)
top_k_ds['label'] = name
top_k_ds

RMSE: 2406.7283
Kendall-thau: 0.4155, (p-value: 0.0000)
##


,top_k,accuracy,recall,precision,f1_score,jaccard_index,label
0,1,0.96,0.000000,0.000000,0.000000,0.000000,wrc_pva_pca20_full_1
1,2,0.92,0.000000,0.000000,0.000000,0.000000,wrc_pva_pca20_full_1
2,3,0.88,0.000000,0.000000,0.000000,0.000000,wrc_pva_pca20_full_1
3,4,0.88,0.250000,0.250000,0.250000,0.142857,wrc_pva_pca20_full_1
4,5,0.84,0.200000,0.200000,0.200000,0.111111,wrc_pva_pca20_full_1
5,6,0.84,0.333333,0.333333,0.333333,0.200000,wrc_pva_pca20_full_1
6,7,0.84,0.428571,0.428571,0.428571,0.272727,wrc_pva_pca20_full_1
7,8,0.80,0.375000,0.375000,0.375000,0.230769,wrc_pva_pca20_full_1
8,9,0.76,0.333333,0.333333,0.333333,0.200000,wrc_pva_pca20_full_1
9,10,0.76,0.400000,0.400000,0.400000,0.250000,wrc_pva_pca20_full_1


In [ ]:
files = ["tls_knn_pca2_full_1",
         ]